## Importing packages

In [42]:
library(here, help, pos = 2, lib.loc = NULL) 
source(here('Modules','modules.R'))
#install_all_packages()
load_library_packages()

Warning message:
"package 'cowplot' was built under R version 3.6.3"Loading required package: rpart
Warning message:
"package 'e1071' was built under R version 3.6.3"Warning message:
"package 'AugmenterR' was built under R version 3.6.3"Warning message:
"package 'smotefamily' was built under R version 3.6.3"Loaded ROSE 0.0-4

Warning message:
"package 'xgboost' was built under R version 3.6.3"
Attaching package: 'xgboost'

The following object is masked from 'package:dplyr':

    slice

Warning message:
"package 'pROC' was built under R version 3.6.3"Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following objects are masked from 'package:stats':

    cov, smooth, var

Warning message in FUN(X[[i]], ...):
"there is no package called 'klaR'"Warning message:
"package 'patchwork' was built under R version 3.6.3"
Attaching package: 'patchwork'

The following object is masked from 'package:cowplot':

    align_plots

The following object is masked from 'package:MASS

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] FALSE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

[[20]]
[1] TRUE

[[21]]
[1] FALSE

[[22]]
[1] TRUE

[[23]]
[1] TRUE

[[24]]
[1] TRUE

[[25]]
[1] FALSE

[[26]]
[1] FALSE

[[27]]
[1] TRUE

[[28]]
[1] TRUE

[[29]]
[1] TRUE

[[30]]
[1] TRUE

[[31]]
[1] TRUE

[[32]]
[1] TRUE

[[33]]
[1] TRUE

[[34]]
[1] FALSE

[[35]]
[1] TRUE

[[36]]
[1] FALSE

[[37]]
[1] TRUE

[[38]]
[1] TRUE

[[39]]
[1] TRUE

[[40]]
[1] TRUE

[[41]]
[1] TRUE

## Importing and Cleaning dataset

In [43]:
df_during <- read_excel(path = here('Data','during_dataset.xlsx'))
df_during <- data.frame(df_during)
unique(sapply(df_during, class))
df_during <- df_during[complete.cases(df_during), ]
dim(df_during)
head(df_during)

[1] "numeric"

[1] 1250   44

PA_barrier_time_family_responsabilities,PA_barrier__tiredness,PA_barrier_places_closed_dangerous,PA_barrier_money,PA_barrier_time_convenience,PA_barrier_hard_work,PA_barrier_no_interest,PA_barrier_none,PA_easiness_outdoor_activities,PA_easiness_professional_guidance,...,soccer,tennis,volleyball,basketball,swimming,surfing,yoga,pilates,fighting,other_sports
0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


### Converting dataset types and labels

- Intesity, duration and frequency >> convert dont practive to zero

In [44]:
df_during$PA_intensity[df_during$PA_intensity == 4] <- 0
df_during$PA_duration[df_during$PA_duration == 5] <- 0
df_during$PA_weekly_frequency[df_during$PA_weekly_frequency == 4] <- 0

- Change practice of exercise boolean logic

In [45]:
df_during$PA_practice_during <- ifelse(df_during$PA_practice_during == 0, 1, 0)

- Change sedentary range to boolean logic

In [46]:
df_during$sedentary_time_range_during[df_during$sedentary_time_range_during < 5] <- 0
df_during$sedentary_time_range_during[df_during$sedentary_time_range_during == 5] <- 1

- Change scholarity range

In [47]:
df_during$scholarity[df_during$scholarity <= 3] <- 1
df_during$scholarity[df_during$scholarity > 3 & df_during$scholarity <= 5] <- 2
df_during$scholarity[df_during$scholarity == 6] <- 3

- Change barrier none label

In [48]:
names(df_during)[8] <- 'PA_barrier_percieve'

- Removing sex others

In [49]:
df_during <- df_during[df_during$sex != 3, ]

### Creating a number of exercises column

In [50]:
sports <- c("running","hiking","cycling",
            "muscle_training","soccer","tennis",
            "volleyball","basketball","swimming",
            "surfing","yoga","pilates","fighting","other_sports")

df_during$PA_number <- rowSums(df_during[, sports])

### Removing Strangers

In [51]:
differ_p <- (   df_during$PA_duration == 0 
                | df_during$PA_intensity == 0 
                | df_during$PA_weekly_frequency == 0
                | df_during$PA_number == 0
            )
strange_p <- df_during$PA_practice_during == 1 & differ_p
df_during <- df_during[!strange_p, ]
table(strange_p)

differ_dp <- ( df_during$PA_duration != 0 
                | df_during$PA_intensity != 0 
                | df_during$PA_weekly_frequency != 0
                | df_during$PA_number != 0
            ) 
strange_dp <- df_during$PA_practice_during == 0 & differ_dp 


table(strange_dp)
df_during <- df_during[!strange_dp, ]

strange_p
FALSE  TRUE 
 1241     6 

strange_dp
FALSE  TRUE 
 1215    26 

## Creting barriers dataset

### Separing Barriers from dataset
<hr />

In [52]:
df_during[1:18] <- lapply(df_during[1:18], as.factor)
df_during_barriers <- df_during[, 1:8]
dim(df_during_barriers)
head(df_during_barriers)

[1] 1215    8

,PA_barrier_time_family_responsabilities,PA_barrier__tiredness,PA_barrier_places_closed_dangerous,PA_barrier_money,PA_barrier_time_convenience,PA_barrier_hard_work,PA_barrier_no_interest,PA_barrier_percieve
2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,1,0
4,0,0,1,0,0,0,0,0
5,0,0,1,0,0,0,0,0
6,1,0,0,0,0,0,0,0
8,0,0,1,0,0,1,0,0


### Agregating barriers 
<hr />

In [53]:

time_barrier_1 <- ifelse(df_during_barriers$PA_barrier_time_family_responsabilities=='0',0,1)
time_barrier_2 <- ifelse(df_during_barriers$PA_barrier_time_convenience=='0',0,1)


hard_barrier_1 <- ifelse(df_during_barriers$PA_barrier__tiredness=='0',0,1)
hard_barrier_2 <- ifelse(df_during_barriers$PA_barrier_hard_work=='0',0,1)


df_during_barriers$PA_barrier_time <- (time_barrier_1 | time_barrier_2) + 0 
df_during_barriers$PA_barrier_hard <- (hard_barrier_1 | hard_barrier_2) + 0

In [54]:
df_during_barriers$PA_barrier_time_family_responsabilities <- NULL
df_during_barriers$PA_barrier_time_convenience <- NULL

df_during_barriers$PA_barrier__tiredness <- NULL
df_during_barriers$PA_barrier_hard_work <- NULL

In [55]:
names(df_during_barriers)

[1] "PA_barrier_places_closed_dangerous" "PA_barrier_money"                  
[3] "PA_barrier_no_interest"             "PA_barrier_percieve"               
[5] "PA_barrier_time"                    "PA_barrier_hard"

## Creating Facilitators dataset

### Separing facilitators from dataset
<hr />

In [56]:
df_during[1:18] <- lapply(df_during[1:18], as.factor)
df_during_facilitators <- df_during[, 9:18]
dim(df_during_facilitators)
head(df_during_facilitators)

[1] 1215   10

,PA_easiness_outdoor_activities,PA_easiness_professional_guidance,PA_easiness_unguided,PA_easiness_music,PA_easiness_collective,PA_easiness_home_activity,PA_easiness_family_activity,PA_easiness_home_space,PA_easiness_home_equipament,PA_easiness_others
2,1,0,0,0,0,0,0,1,1,0
3,0,0,1,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,1,1,0
5,0,0,0,0,0,1,1,1,1,0
6,1,0,0,0,0,0,0,0,1,0
8,0,1,0,1,0,0,1,0,1,0


### Agregating facilitators
<hr />

In [57]:

home_facilitator_1 <- ifelse(df_during_facilitators$PA_easiness_home_activity=='0',0,1)
home_facilitator_2 <- ifelse(df_during_facilitators$PA_easiness_home_equipament=='0',0,1)
home_facilitator_3 <- ifelse(df_during_facilitators$PA_easiness_home_space=='0',0,1)


group_easiness_1 <- ifelse(df_during_facilitators$PA_easiness_family_activity=='0',0,1)
group_easiness_2 <- ifelse(df_during_facilitators$PA_easiness_collective=='0',0,1)



df_during_facilitators$PA_facilitator_home <- (home_facilitator_1 | home_facilitator_2 | home_facilitator_3) + 0 
df_during_facilitators$PA_facilitator_group<- (group_easiness_1 | group_easiness_2) + 0


In [58]:
df_during_facilitators$PA_easiness_home_activity <- NULL
df_during_facilitators$PA_easiness_home_equipament <- NULL
df_during_facilitators$PA_easiness_home_space <- NULL
df_during_facilitators$PA_easiness_family_activity <- NULL
df_during_facilitators$PA_easiness_collective <- NULL
df_during_facilitators$PA_easiness_unguided <- NULL

In [59]:
names(df_during_facilitators)

[1] "PA_easiness_outdoor_activities"    "PA_easiness_professional_guidance"
[3] "PA_easiness_music"                 "PA_easiness_others"               
[5] "PA_facilitator_home"               "PA_facilitator_group"